<a href="https://colab.research.google.com/github/ligerre/firsttime/blob/main/stopping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import pickle

In [ ]:
class FRBS:
    def __init__(self,f,z,beta,eta):
        self._z = z
        self._beta = beta
        self._eta = eta*beta/2
        self._f = f
        #store gradient
        self.past_z = self._f.F_operator(z)
        #store average iterate
        self._z_avg = torch.stack([torch.zeros(d,dtype=torch.double).to(device),torch.zeros(d,dtype=torch.double).to(device)])
        self.cnt =0
    def step(self):

        #calculate current gradient
        z_grad = self._f.F_operator(self._z)
        #calculate increasement z_k+1 = z_k - eta/beta((1+beta)F(z_k)-F(z_k-1))
        self._z = self._z - (self._eta/self._beta)*((1+self._beta)*z_grad-self.past_z)
        #update past gradient
        self.past_z = z_grad
        #update average iteration
        self.cnt+=1
        self._z_avg = self._z_avg+self._z
        #getting return values
        average_val = self._f.eval(self._z_avg[0]/self.cnt,self._z_avg[1]/self.cnt)
        return average_val

In [ ]:
class PEG:
    def __init__(self,f,z,beta,eta):
        self._z = z
        self._beta = beta
        self._eta = eta*beta
        self._f = f
        #store average iterate
        self._z_avg = torch.stack([torch.zeros(d,dtype=torch.double).to(device),torch.zeros(d,dtype=torch.double).to(device)])
        self.cnt =0
    def step(self):
        #taking half a step z_k_1/2 = z_k - eta/beta F(z_k)
        z_half = self._z - (self._eta/self._beta)*self._f.F_operator(self._z)
        #calculate increasement z_k+1 = z_k - eta F(z_k_1/2)
        self._z = self._z - self._eta*self._f.F_operator(z_half)
        #update average iteration
        self.cnt+=1
        self._z_avg = self._z_avg+self._z
        #getting return values
        average_val = self._f.eval(self._z_avg[0]/self.cnt,self._z_avg[1]/self.cnt)
        return average_val

In [ ]:
class Bilinear:
    def __init__(self,A,h,b):
        self._A = A
        self._H = 2*torch.matmul(torch.t(self._A),self._A)
        self._h = h
        self._b = b
    def x_div(self,x,y):
        return torch.matmul(self._H,x)-self._h - torch.matmul(torch.t(self._A),y)
    def y_div(self,x,y): #negative gradient at y
        return torch.matmul(self._A,x)-self._b
    def F_operator(self,z):
        return torch.stack([self.x_div(z[0],z[1]),self.y_div(z[0],z[1])])
    def eval(self,x,y):
        res = 0.5* torch.dot(x,torch.matmul(self._H,x))-torch.dot(x,self._h)-torch.dot(y,torch.matmul(self._A,x)-self._b)
        return res.item()

In [ ]:
def get_dict(file_name):
    try:
        with open(file_name,'rb') as f:
            result = pickle.load(f)
    except:
        result = {}
    return result
def save_dict(dict,file_name):
    with open(file_name, 'wb') as f:
        pickle.dump(dict, f)

In [ ]:
def get_starting_vector(x_star,y_star,dis):
    z_star = torch.cat([x_star.clone(),y_star.clone()])
    l = torch.rand(2*d).to(device)
    z_0= z_star + (dis/torch.linalg.vector_norm(l).item())*l
    z_0 = torch.tensor_split(z_0,[d])

    return z_0[0],z_0[1]

In [ ]:
def run_FBRS(beta,eta):
  res = 0
  print("FBRS",' ',beta)
  for i in range(10):
    x_0,y_0 = get_starting_vector(x_star,y_star,dis)
    z_0 = torch.stack([x_0,y_0])
    optimizer = FRBS(f,z_0,beta,eta)
    avg = []
    while True:
        avg_res = optimizer.step()
        avg.append(abs(avg_res-val))
        if len(avg)>1000:
            avg.pop(0)
            if max(avg)<tol:
                break
    print(optimizer.cnt)
    res +=optimizer.cnt
  FBRS_list[(beta,eta/2)]=res/10

In [ ]:
def run_PEG(beta,eta):
  print("PEG",' ',beta)
  res = 0
  for i in range(10):
    x_0,y_0 = get_starting_vector(x_star,y_star,dis)
    z_0 = torch.stack([x_0,y_0])
    optimizer = PEG(f,z_0,beta,eta)
    avg = []
    while True:
        avg_res = optimizer.step()
        avg.append(abs(avg_res-val))
        if len(avg)>1000:
            avg.pop(0)
            if max(avg)<tol:
                break
    print(optimizer.cnt)
    res +=optimizer.cnt

  PEG_list[(beta,eta)]=res/10

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
d = 1000
dis = 10
tol = 10**(-6)
#generate vector A
diag = np.ones(d)
subdiag = -np.ones(d-1)
A = (np.diag(diag)+np.diag(subdiag,1))*0.25
A = np.fliplr(A).copy()
A = torch.from_numpy(A).to(device)
#generate b and h
b = 0.25*torch.ones(d,dtype=torch.double).to(device)
h = np.zeros(d)
h[d-1] = 1
h = h*0.25
h = torch.from_numpy(h).to(device)
#generate the true solution
x_star=torch.tensor([i+1. for i in range(d)],dtype=torch.double).to(device)
y_star = -0.5*torch.ones(d,dtype=torch.double).to(device)


f=Bilinear(A,h,b)
val=f.eval(x_star,y_star)

FBRS_test = "FBRS1000.txt"
PEG_test = "PEG1000.txt"

FBRS_list = get_dict(FBRS_test)
PEG_list = get_dict(PEG_test)

Using device: cuda


In [ ]:
betas = [1.0,0.85,0.75,0.6,0.5,0.32,0.25]
etas = [0.8,0.5]
for beta in betas:
  for eta in etas:
    run_FBRS(beta,eta)
    run_PEG(beta,eta)
print(FBRS_list)
print(PEG_list)

FBRS   1.0
80257
80215
80217
80224
80207
80206
80281
80207
80188
80219
PEG   1.0
60424
60421
60403
60421
60398
60430
60429
60375
60429
60416
FBRS   1.0
64316
64506
64325
64423
64408
64434
64336
64389
64457
64371
PEG   1.0
64376
64377
64403
64389
64393
64368
64375
64390
64371
64385
FBRS   0.85
94242
94259
94238
94248
94244
94188
94165
94194
94228
94258
PEG   0.85
70904
70910
70911
70889
70902
70880
70882
70918
70892
70912
FBRS   0.85
75555
75653
75586
75561
75500
75441
75529
75627
75613
75610
PEG   0.85
75538
75600
75528
75560
75556
75545
75552
75531
75611
75563
FBRS   0.75
53835
53842
53827
53830
53849
53797
53826
53816
53806
53851
PEG   0.75
53831
53820
53791
53834
53807
53844
53833
53842
53820
53808
FBRS   0.75
85548
85457
85560
85538
85578
85479
85583
85517
85545
85409
PEG   0.75
85478
85526
85558
85458
85493
85543
85505
85537
85489
85543
FBRS   0.6
67055
66999
67033
67121
66943
67073
67051
66992
67011
67072
PEG   0.6
66980
67025
67030
67001
67003
67013
67043
67032
67036
67043
FBRS 

In [ ]:
save_dict(FBRS_list,FBRS_test)
save_dict(PEG_list,PEG_test)